In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# File path for carbon intensity dataset
PATH = "C:/Users/Krist/OneDrive/Documents/Data Analysis/Practice/Carbon/data/processed/df_carbon.parquet"

# Keeps the DATETIME column as an index
df_carbon = pd.read_parquet(PATH)

# Sort Ascending order by index (DATETIME)
df_carbon = df_carbon.sort_index()

# Keep records within the modern period due to less fluctuation in data readings (determined in Notebook 02)
df_carbon = df_carbon[df_carbon.index >= "2020-01-01"]

# Base high-usage UK household profile (14kwh/day)

In [3]:
# Base high-usage UK household profile (14 kWh/day)
HOUSEHOLD_PROFILE_RAW = np.array([
    0.25, 0.23, 0.22, 0.22, 0.25,  # 00–04
    0.35, 0.55, 0.65, 0.60,        # 05–08
    0.55, 0.50, 0.48, 0.47, 0.50, 0.55,  # 09–14
    0.60, 0.75, 1.10, 1.20, 1.05,  # 15–19 (evening peak)
    0.70, 0.55, 0.40, 0.30         # 20–23
])

def make_household_profile(daily_kwh: float = 14.0) -> np.ndarray:
    """
    Return a 24-element array of hourly kWh for a single day,
    scaled so the total equals daily_kwh.
    """
    raw = HOUSEHOLD_PROFILE_RAW.copy()
    scale = daily_kwh / raw.sum()
    return raw * scale

A generic function that can:

- Take a carbon intensity series (24 hourly values)
- Optionally a renewable share series (for the “renewables” strategy)
- A flexible share (0–0.5)
- A strategy: "low_intensity" or "max_renewable"

Returns:
- baseline load + emissions
- shifted load + emissions
- summary stats

In [5]:
def compute_renewable_share(df_day: pd.DataFrame) -> pd.Series:
    """
    Compute renewable share (0–1) from RENEWABLE and GENERATION for a given day slice.
    """
    share = df_day["RENEWABLE"] / df_day["GENERATION"]
    return share.clip(lower=0.0, upper=1.0)

In [7]:
def run_shift_scenario(
    ci_series: pd.Series,
    daily_kwh: float = 14.0,
    flexible_share: float = 0.3,
    strategy: str = "low_intensity",
    renewable_share: pd.Series | None = None,
) -> dict:
    # Ensure sorted and 24 hours
    ci_series = ci_series.sort_index()
    assert len(ci_series) == 24, "ci_series must contain exactly 24 hourly values"

    if strategy == "max_renewable":
        if renewable_share is None:
            raise ValueError("renewable_share is required for 'max_renewable' strategy")
        renewable_share = renewable_share.loc[ci_series.index].sort_index()

    # Build baseline load profile
    baseline_load = make_household_profile(daily_kwh)  # kWh per hour
    index = ci_series.index

    # Split into non-flexible and flexible components
    baseline_load = pd.Series(baseline_load, index=index)
    nonflex_load = baseline_load * (1.0 - flexible_share)
    flex_load = baseline_load * flexible_share

    total_flex_energy = flex_load.sum()

    # Decide destination hours (where to move flex usage)
    if strategy == "low_intensity":
        # Sort hours by ascending carbon intensity
        target_order = ci_series.sort_values(ascending=True).index
    elif strategy == "max_renewable":
        # Sort hours by descending renewable share
        target_order = renewable_share.sort_values(ascending=False).index
    else:
        raise ValueError("strategy must be 'low_intensity' or 'max_renewable'")

    # Allocate flexible energy into target hours
    shifted_flex = pd.Series(0.0, index=index)
    remaining_energy = total_flex_energy

    for ts in target_order:
        if remaining_energy <= 0:
            break
        # For now: we allow arbitrary capacity per hour, just spread evenly
        # You can impose caps later if you want.
        # Simple strategy: fill up to original flex_load[ts] + some factor, or just distribute evenly.
        # Here: just proportionally spread over sorted hours.
        # We'll divide remaining energy equally over all remaining target slots.
        hours_left = (target_order == ts).sum()  # wrong: so simplify instead
        # To keep it simple: assign a fixed chunk
        chunk = total_flex_energy / len(target_order)
        assign = min(chunk, remaining_energy)
        shifted_flex[ts] += assign
        remaining_energy -= assign

    # If numerical leftovers remain, dump them into the best hour
    if remaining_energy > 1e-6:
        best_ts = target_order[0]
        shifted_flex[best_ts] += remaining_energy
        remaining_energy = 0.0

    # Build final shifted load
    shifted_load = nonflex_load + shifted_flex

    # Emissions (gCO2) = kWh * gCO2/kWh
    ci_array = ci_series.values
    baseline_emissions = baseline_load.values * ci_array
    shifted_emissions = shifted_load.values * ci_array

    total_baseline_emissions = baseline_emissions.sum()
    total_shifted_emissions = shifted_emissions.sum()

    relative_reduction = (
        (total_baseline_emissions - total_shifted_emissions) / total_baseline_emissions
    )

    return {
        "index": index,
        "ci": ci_array,
        "baseline_load": baseline_load.values,
        "shifted_load": shifted_load.values,
        "baseline_emissions": baseline_emissions,
        "shifted_emissions": shifted_emissions,
        "total_baseline_emissions": total_baseline_emissions,
        "total_shifted_emissions": total_shifted_emissions,
        "relative_reduction": relative_reduction,
    }

Note: the allocation loop is intentionally simple. It can be redefined later (e.g., allocate equal proportion into lowest-intensity hours only, or cap hourly shifted load).

# Historical mode

In [ ]:
def get_day_slice(df: pd.DataFrame, date: str) -> pd.DataFrame:
    """
    Return a one-day slice for a given date string 'YYYY-MM-DD'.
    """
    return df.loc[date : date]  # inclusive, hourly data


# Example: any historical day
date = "2024-02-05"
df_day = get_day_slice(df_carbon, date)

ci_day = df_day["CARBON_INTENSITY"]
renewable_share_day = compute_renewable_share(df_day)

scenario_low = run_shift_scenario(
    ci_series=ci_day,
    daily_kwh=14.0,
    flexible_share=0.3,
    strategy="low_intensity",
    renewable_share=renewable_share_day,
)

scenario_renew = run_shift_scenario(
    ci_series=ci_day,
    daily_kwh=14.0,
    flexible_share=0.3,
    strategy="max_renewable",
    renewable_share=renewable_share_day,
)

# Forecast mode